In [1]:
import os
import sys
sys.path.append('..')
from unstructured.staging.base import elements_from_json
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from src.utils.paths import get_project_path

In [2]:
def load_processed_files(directory_path: str) -> list:
    elements = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            try:
                elements.extend(elements_from_json(filename=file_path))
            except IOError:
                print(f"Error: Could not read file {filename}.")

    return elements

elements = load_processed_files(os.path.join(get_project_path(), 'data', 'parsed_pages'))

In [ ]:
documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    documents.append(Document(page_content=element.text, metadata=metadata))

In [ ]:
db = FAISS.from_documents(
    documents,
    HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
)

In [ ]:
db.save_local(os.path.join(get_project_path(), "db", "intfloat_multilingual-e5-large"))